In [1]:
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import urllib.request
import requests
import pandas as pd
import re
import csv

def crawl_page(link=''):
    # req = urllib.request.Request(
    #         link, 
    #         data=None, 
    #         headers={
    #             'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    #             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393'
    #         }
    # )
    # try:
    #     page= urllib.request.urlopen(req)
    #     return_code= page.getcode()
    #     soup_page= BeautifulSoup(page,'html.parser')
    #     return {'page':soup_page, 'status': return_code}
    # except HTTPError as err:
    #     if err.code == 404:
    #         return_code=404
    #         return {'page':'no page', 'status': return_code}
    #     else:
    #         raise
    try:
        response = requests.get(
            url='http://api.scraperapi.com', #scrapeapi
            params={
                'api_key': 'b4b2907633a4c7758d3e81f40c85a442',
                'url': link,  
            },
            
        )
        return_code= 200
        soup_page= BeautifulSoup(response.content,'html.parser')
        return {'page':soup_page, 'status': return_code}
    except requests.exceptions.HTTPError as err:
        if err.response.status_code == 404:
            return_code=404
            return {'page':'no page', 'status': return_code}
    

def write_csv(line=[], file=''):
    with open(file, 'a', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=' ',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow(line)

def remove_row_csv():
    with open('temp_27.csv') as inp:
        for link in csv.reader(inp):
            count=0
            temp=link
            with open('used_estate.txt') as out:
                for estate in csv.reader(out):
                    if re.match("(.*)/"+estate[0],temp[1]) is not None:
                        count+=1
            if count==0:
                new_row= [link[0]+','+link[1]]
                write_csv(new_row,"temp_28.csv")

def create_urls(file=''):
    with open(file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 1
        for row in csv_reader:
            new_row= [str(line_count)+','+row[0]]
            write_csv(new_row,'property_url_v2.csv')
            line_count+=1

    

In [17]:
class DataCollect():
    def __init__(self):
        self.__url_to_open='https://www.property24.com'
        self.__soup=crawl_page(self.__url_to_open)['page']
        self.__area_url=[]
        self.__list_url=[]
        self.__urls=pd.read_csv("temp_17_v5.csv",usecols= ['id','url']) 
    
    def collect_all_first_page_url(self):
        other_links= self.__soup.find('div',class_='p24_popular').find(class_='col-8').find_all('a',class_='p24_bold')
        for link in other_links:
            self.__area_url.append(self.__url_to_open+link.get('href'))
        first_element_tag= self.__soup.find('div',class_='p24_popular').find(class_='col-8').find_all(class_='row')[1]
        first_link=first_element_tag.find('a').get('href')
        temp_soup= crawl_page(self.__url_to_open+first_link)['page']
        parent_link= temp_soup.find(id='breadCrumbContainer').find_all('a')[1]
        self.__area_url.append(self.__url_to_open+parent_link.get('href'))
    
    def collect_all_url(self):
        self.collect_all_first_page_url()
        for link in self.__area_url:
            soup= crawl_page(link)['page']
            link_house_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("houses-for-sale")).get('href')
            link_apart_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("apartments-for-sale")).get('href')
            link_townhouse_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("townhouses-for-sale")).get('href')
            self.__list_url.append(link_house_tag)
            self.__list_url.append(link_apart_tag)
            self.__list_url.append(link_townhouse_tag)

    def get_property_data(self, link=''):
        data={}

        estate_id= link.split('/')[-1].split('?')[0]

        prop_soup_element= crawl_page(self.__url_to_open+link)

        
        if prop_soup_element['status'] !=200:
            print('heres')
            return False
        
        prop_soup= prop_soup_element['page']

        if prop_soup is None:
            print('here')
            return False

        #check if page not found
        check_page_not_found_msg= prop_soup.select_one('.p24_not_found')
        if check_page_not_found_msg is not None:
            print("====> Page not found !!")
            return False
        
        #check page expired
        check_page_expired = prop_soup.select_one('.p24_results.p24_expired')
        if check_page_expired is not None:
            print("====> Page expired !!")
            return False
        

        name= prop_soup.find('div',class_='p24_listingFeaturesWrapper').find('h1').get_text()
        data['name']=name

        price= prop_soup.find('div',class_='p24_listingFeaturesWrapper').find('div',class_="p24_price").get_text()
        data['price']=price

        province= link.split('/')[4]
        data['province']=province

        city= link.split('/')[3]
        data['city']=city


        is_adress= len(prop_soup.find('div',class_='p24_listingFeaturesWrapper').find_all('div',class_="p24_mBM")) >= 3
        if is_adress:
            if len(prop_soup.find('div',class_='p24_listingFeaturesWrapper').find_all('div',class_="p24_mBM")) == 3:
                index=-1
            else:
                index=3
            address= prop_soup.find('div',class_='p24_listingFeaturesWrapper').find_all('div',class_="p24_mBM")[index].get_text()
            data['address']=address
        else:
            address=  prop_soup.select_one('.p24_addressPropOverview').get_text()
            data['address']=address

        #property overview
        if type(prop_soup.select_one('.p24_propertyOverview')) is not None:
            prop_overview_panel= prop_soup.find('div',class_='p24_propertyOverview').find('div',class_='panel-body')
            prop_overview_row= prop_overview_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #rooms
        rooms_panel=prop_soup.select_one('#js_accordion_rooms')
        if rooms_panel is not None:
            prop_overview_row= rooms_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #external_features
        external_features_panel=prop_soup.select_one('#js_accordion_externalfeatures')
        if external_features_panel is not None:
            prop_overview_row= external_features_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value
        
        #other_features
        other_features_panel=prop_soup.select_one('#js_accordion_otherfeatures')
        if other_features_panel is not None:
            prop_overview_row= other_features_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #building_features
        building_features_panel=prop_soup.select_one('#js_accordion_building')
        if building_features_panel is not None:
            prop_overview_row= building_features_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #points_interest
        interest_soup= crawl_page(self.__url_to_open+'/ListingReadOnly/PointsOfInterestForListing?ListingNumber=P24-'+estate_id)
        # points_interest_panel=prop_soup.select_one('#accordian-points-of-interest')
        # print(points_interest_panel)
        if interest_soup['status'] == 200:
            categories= interest_soup['page'].select('.js_P24_POICategory')
            for cat in categories:
                interest= cat.select_one('.row > .col-6 > .p24_semibold')
                distances= cat.select('.row > .col-6.noPadding.p24_semibold')
                dist_tab=[]
                for dist in distances:
                    dist_tab.append(dist.get_text())
                data[interest.get_text()]=dist_tab

        print(str(data))
        write_csv([str(data)],'data_property_v2.csv')

        
        return True

    def crawl_property_per_page(self,city,page,number_properties):
        link=self.__url_to_open+city+'/p'+str(page)
        print('==>Page link: ',link)
        cur_soup= crawl_page(link)['page']
        properties= cur_soup.find_all('div',class_='js_resultTile')
        for prop_index in range(len(properties)-1):
            prop_link= properties[prop_index].find('a').get('href')
            write_csv([prop_link],'data_property_url.csv')

        number_properties+= len(properties)
        return number_properties

    def crawl_property_per_city(self,city,cur_page):
        soup= crawl_page(self.__url_to_open+city)['page']
        pagination= soup.find('div',class_='p24_pager').find_all('li')[-1].find('a').get('data-pagenumber')
        page_number= int(pagination)
        number_properties=0
        print("City url: ",city)
        for page in range(cur_page, page_number+1):
            print("=>Page number: ",page)
            number_properties = self.crawl_property_per_page(city,page,number_properties)
        print(number_properties)
    
    def crawl_properties(self):
        #shuffle
        self.__urls = self.__urls.sample(frac = 1).reset_index(drop=True)
        for i in range(self.__urls.size):
            prop_link= self.__urls.iloc[i]['url']
            
            print('==>Property link: ', prop_link)
            check_property_crawl = self.get_property_data(prop_link)
            if check_property_crawl:
                self.__urls= self.__urls.drop(self.__urls.index[i])
                self.__urls.to_csv('temp_18_v5.csv',index=False)

    def crawl_property_url(self):
        self.__list_url=['/townhouses-for-sale/gauteng/1']
        for link in self.__list_url:
            self.crawl_property_per_city(link,1)

In [18]:
dc= DataCollect()
try:
    dc.crawl_properties()
except:
    dc.crawl_properties()

==>Property link:  /for-sale/savoy-estate/johannesburg/gauteng/5287/109181147
{'name': '2 Bedroom Apartment / Flat for Sale in Savoy Estate', 'price': 'R 650 000', 'province': 'gauteng', 'city': 'johannesburg', 'address': 'Savoy Estate, Johannesburg', 'Listing Number': '109181147', 'Type of Property': 'Apartment / Flat', 'Listing Date': '01 October 2020', 'Floor Size': '98 m²', 'Price per m²': 'R 6 633', 'Levies': 'R 1 760', 'Rates and Taxes': 'R 260', 'Bedrooms': '2', 'Bedroom 1': 'Bedroom 2', 'Bedroom 2': 'Bedroom 1', 'Bathroom': 'Bathroom', 'Kitchen': 'Kitchen', 'Reception Rooms': '1', 'Lounge': 'Lounge', 'Dining Room': 'Dining Room', 'Entrance Hall': 'Entrance Hall', 'Garage': 'Garage', 'Garden': 'Garden', 'Health': ['0.37km'], 'Education': ['0.75km', '0.78km', '0.81km', '1.11km', '1.17km', '1.37km', '1.48km', '1.51km', '1.59km', '1.73km', '2.00km', '2.04km', '2.04km', '2.09km', '2.12km', '2.14km', '2.21km'], 'Transport and Public Services': ['0.93km', '1.01km', '1.11km', '1.47km',

{'name': '0.5 Bedroom Apartment / Flat for Sale in Arcadia', 'price': 'R 300 000', 'province': 'gauteng', 'city': 'pretoria', 'address': '\n0.5 Bedroom Apartment / Flat for Sale in Arcadia\n', 'Listing Number': '110183021', 'Type of Property': 'Apartment / Flat', 'Street Address': ' 485 TREVENNA STREET, Arcadia', 'Listing Date': '09 July 2021', 'Floor Size': '39 m²', 'Price per m²': 'R 7 692', 'Levies': 'R 1 300', 'Rates and Taxes': 'R 100', 'Bedroom': 'Tiled Floors, Built-in Cupboards', 'Bathroom': 'Tiled Floors, Bath, Toilet and Basin', 'Kitchen': 'Stove, Tiled Floors, Built-in Cupboards', 'Nearby Public Transport': 'Bus, Minibus Taxi', 'Wheelchair Accessible': 'Yes', 'Sports and Leisure': ['0.18km'], 'Health': ['0.26km', '0.37km', '0.40km', '0.45km', '0.48km', '0.56km'], 'Food and Entertainment': ['0.28km', '0.29km', '0.31km', '0.32km', '0.34km', '0.39km', '0.41km', '0.43km', '0.45km', '0.49km', '0.56km', '0.59km'], 'Shopping': ['0.34km', '0.42km', '0.48km', '0.49km', '0.49km', '0.5

{'name': '3 Bedroom Townhouse for Sale in Golden Acres Estate', 'price': 'R 2 450 000', 'province': 'limpopo', 'city': 'tzaneen', 'address': 'Golden Acres Estate, Tzaneen', 'Listing Number': '109988099', 'Type of Property': 'Townhouse', 'Listing Date': '18 May 2021', 'Erf Size': '539 m²', 'Rates and Taxes': 'R 900', 'Bedrooms': '3', 'Bedroom 1': 'Tiled Floors, Built-in Cupboards', 'Bedroom 2': 'Tiled Floors, Built-in Cupboards', 'Bedroom 3': 'Tiled Floors, Built-in Cupboards', 'Bathrooms': '3', 'Bathroom 1': 'Tiled Floors, Main en Suite, Shower, Bath, Toilet and Basin', 'Bathroom 2': 'Tiled Floors, Shower, En suite, Bath, Toilet and Basin', 'Bathroom 3': 'Tiled Floors, Bath, En suite, Shower, Toilet and Basin', 'Kitchen': 'Scullery, Laundry, Tiled Floors', 'Lounge': 'Tiled Floors', 'Dining Room': 'Tiled Floors', 'Other': 'Patio', 'Garage': '2', 'Gardens': '1', 'Security': 'Totally Fenced', 'Roof': 'Tile', 'Wall': 'Plaster', 'Education': ['1.67km', '1.77km', '2.85km', '3.21km', '3.34km'

{'name': '3 Bedroom House for Sale in Silverfields', 'price': 'R 1 350 000', 'province': 'gauteng', 'city': 'krugersdorp', 'address': '\n3 Bedroom House for Sale in Silverfields\n', 'Listing Number': '110252894', 'Type of Property': 'House', 'Street Address': '5 Orchid street, Silverfields', 'Listing Date': '31 July 2021', 'Erf Size': '992 m²', 'Rates and Taxes': 'R 1 300', 'Pets Allowed': 'Yes', 'Zoning': 'Single Residential', 'Bedrooms': '3', 'Bathrooms': '2', 'Kitchens': '1', 'Office/studies': '2', 'Domestic Rooms': '1', 'Lounges': '1', 'Dining Rooms': '1', 'Garage': '2', 'Garden': 'Yes', 'Pool': 'Yes', 'Special Feature': 'Communal braai area', 'Transport and Public Services': ['0.74km', '1.47km', '2.98km', '3.35km', '3.36km'], 'Education': ['1.08km', '1.16km', '2.02km', '2.22km', '2.28km', '2.28km', '2.39km', '2.49km', '2.65km', '2.97km', '3.01km', '3.15km', '3.21km', '3.32km', '3.33km', '3.66km', '3.76km', '3.86km', '3.99km', '4.31km', '4.41km'], 'Food and Entertainment': ['2.07km

{'name': '7 Bedroom House for Sale in Haenertsburg', 'price': 'R 3 830 000', 'province': 'limpopo', 'city': 'tzaneen', 'address': 'Haenertsburg, Tzaneen', 'Listing Number': '107005376', 'Type of Property': 'House', 'Listing Date': '23 March 2021', 'Erf Size': '971 m²', 'Levies': 'R 1 500', 'Rates and Taxes': 'R 1 900', 'Pets Allowed': 'No', 'Bedrooms': '7', 'Bathrooms': '5', 'Kitchens': '2', 'Garage': '4', 'Education': ['0.37km', '2.04km', '3.11km', '5.14km', '7.15km', '7.74km', '7.90km', '9.01km', '9.34km', '9.66km', '10.47km', '11.29km', '13.16km'], 'Transport and Public Services': ['4.26km']}
==>Property link:  /for-sale/camps-bay/cape-town/western-cape/11014/108227958
{'name': '5 Bedroom House for Sale in Camps Bay', 'price': 'R 29 995 000', 'province': 'western-cape', 'city': 'cape-town', 'address': 'Camps Bay, Cape Town', 'Listing Number': '108227958', 'Type of Property': 'House', 'Description': 'Multi Storey', 'Listing Date': '28 November 2019', 'Erf Size': '486 m²', 'Floor Size

AttributeError: 'NoneType' object has no attribute 'find'